# DFTB SCC Calculator for H20 Molecule

# Setting up the enviroment

In [2]:
import torch
from tbmalt import Geometry, OrbitalInfo
from tbmalt.tools.downloaders import download_dftb_parameter_set
from tbmalt.physics.dftb import Dftb2
from tbmalt.physics.dftb.feeds import SkFeed, SkfOccupationFeed, HubbardFeed, RepulsiveSplineFeed
from tbmalt.data.units import length_units


torch.set_default_dtype(torch.float64)

# Input the molecular system

In [3]:
def H2O_scc(device):
    # Ensure device is a torch device
    if isinstance(device, str):
        device = torch.device(device)
    
    cutoff = torch.tensor([9.98], device=device)

    geometry = Geometry(
        torch.tensor([8,1,1], device=device),
        torch.tensor([
            [0.00000000, -0.71603315, -0.00000000],
            [0.00000000, -0.14200298, 0.77844804 ],
            [-0.00000000, -0.14200298, -0.77844804]],
            device=device),units='a')

    orbs = OrbitalInfo(geometry.atomic_numbers, {1: [0], 8: [0, 1]})

    return geometry, orbs 


# Loading of the DFTB parameters into their associated feed objects

In [4]:
def feeds_scc(device, skf_file):
    species = [1, 8]
    h_feed = SkFeed.from_database(skf_file, species, 'hamiltonian', device=device)
    s_feed = SkFeed.from_database(skf_file, species, 'overlap', device=device)
    o_feed = SkfOccupationFeed.from_database(skf_file, species, device=device)
    u_feed = HubbardFeed.from_database(skf_file, species, device=device)
    r_feed = RepulsiveSplineFeed.from_database(skf_file, species, device=device)

    return h_feed, s_feed, o_feed, u_feed, r_feed

# Constructing the SCC-DFTB calculator

In [5]:
file_path = "mio.h5"  # Save as HDF5 file
device = torch.device('cpu')

h_feed, s_feed, o_feed, u_feed, r_feed = feeds_scc(device,file_path)

calculator = Dftb2(h_feed, s_feed, o_feed, u_feed, r_feed)

geometry, orbs = H2O_scc(device)

# Performing Calculations

In [8]:
energy = calculator(geometry, orbs)
print('Energy:', energy)

total_energy = calculator.total_energy
print('Total energy:', total_energy)

q_final_atomic = calculator.q_final_atomic
print('Q Final Atomic:', q_final_atomic)

hamiltonian = calculator.hamiltonian
print('Hamiltonian:', hamiltonian)

forces = calculator.forces
print('forces:', forces)

dipole = calculator.dipole
print('dipole_moment:', dipole)

Energy: tensor([0.0910])
Total energy: tensor([-4.0779])
Q Final Atomic: tensor([6.5926, 0.7037, 0.7037])
Hamiltonian: tensor([[-0.8059,  0.0000,  0.0000,  0.0000, -0.4759, -0.4759],
        [ 0.0000, -0.2592,  0.0000,  0.0000, -0.1917, -0.1917],
        [ 0.0000,  0.0000, -0.2592,  0.0000, -0.2599,  0.2599],
        [ 0.0000,  0.0000,  0.0000, -0.2592,  0.0000,  0.0000],
        [-0.4759, -0.1917, -0.2599,  0.0000, -0.2276, -0.1049],
        [-0.4759, -0.1917,  0.2599,  0.0000, -0.1049, -0.2276]])
forces: tensor([[ 1.2326e-32, -8.3752e-06, -7.6328e-17],
        [-2.4312e-17,  4.1876e-06,  1.9991e-05],
        [ 2.4312e-17,  4.1876e-06, -1.9991e-05]])
dipole_moment: tensor([ 0.0000e+00, -6.4284e-01, -3.3307e-16])


/home/user/venvs/bachelor/lib/python3.11/site-packages/tbmalt/physics/dftb/__init__.py:1140: UserWarning: This is an experimental feature which is not intended for public use!
  warnings.warn(
